In [ ]:
import torch
import os
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from skimage import io
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#total images in dataset = 1540
#for each image 35 divisions made, and in total 44 images, so 35*44 = 1540 
#after excluding 1.0.0 , 2.0.0 , 3.0.0 , 4.0.0 we will have 140 images less
#so in total  = 1400 images
#80% of 1400 = 1120

from torch.utils.data import Dataset
import torchvision.transforms as tf
import os
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm

class MedicalImage(Dataset):
  
  def __init__(self, data_path):
    self.path = data_path
    all_files = os.listdir(data_path)
    l = len(all_files)
    #self.lof=[]
    self.val=[]
    count=0
    for item in all_files:
      if item.split("_")[1]!='0':
        if(count>0.727*l): # since 0.727*1540 = 1120
          self.val.append(item) # 240
          #pass
        else:
          count+=1
          #self.lof.append(item) # 1120
    print(len(self.val))

  def __len__(self):
    #x = len(self.lof)
    y = len(self.val)
    return (y)

  def __getitem__(self,index):
    #img_path = f'{self.path}{self.lof[index]}'
    val_path = f'{self.path}{self.val[index]}'
    #img = Image.open(img_path)
    val_img = Image.open(val_path)
    #z = self.lof[index].split('_')[2]
    z1 = self.val[index].split('_')[2]
    #if (z == 0 or z1 == 0):
    if (z1==0):
      sf = 2.5
    else:
      sf = 10.0
    #print(img.size)
    #print(val_img.size)
    #img = img.resize((256,256),resample=Image.BILINEAR)
    val_img = val_img.resize((256,256),resample=Image.BILINEAR)
    #w,h = img.size
    w1,h1 = val_img.size #validation
    
    blur_val_img = val_img.resize((int(w1/sf),int(h1/sf)),resample=Image.BILINEAR) #validation
    blur_img = blur_val_img.resize((w1,h1),resample=Image.BILINEAR) #validation

    #blur_img = img.resize((int(w/sf),int(h/sf)),resample=Image.BILINEAR)
    #blur_img = blur_img.resize((w,h),resample=Image.BILINEAR)
    
    input_val_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(blur_val_img)) #validation
    target_val_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(val_img)) #validation
    
    #input_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(blur_img))
    #target_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(img))
    #return(input_img, target_img, input_val_img, target_val_img )
    return(input_val_img, target_val_img)

data = MedicalImage('/content/drive/MyDrive/Medical Image Dataset/')  # PRETAM'S MACHINE
#data = MedicalImage('/content/drive/MyDrive/Cropped Medical Images/')  # SWARNENDU'S MACHINE
inp,tar = data[34]
print(inp.shape, tar.shape)

280
torch.Size([3, 25, 25]) torch.Size([3, 256, 256])


In [ ]:
# Hyperparameters
LEARNING_RATE = 2e-4 
BATCH_SIZE = 48
IMAGE_SIZE = 256
CHANNELS_IMG = 3
IN_CHANNNELS = 3
OUT_CHANNELS = 8
NUM_EPOCHS = 20
FEATURES_DISC = 16
FEATURES_GEN = 16

In [ ]:
#Load Data
from torch.utils.data import DataLoader

trainloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

testloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=False)

NameError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

**Generator Block**

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,IC,OC,K,S,P):
    super(ConvBlock, self).__init__()
    self.conv = nn.Conv2d(IC,OC,K,S,P)
    self.bn = nn.BatchNorm2d(OC)
    self.relu = nn.LeakyReLU()
    self.conv1 = nn.Conv2d(OC,OC,K,S,P)
    self.bn1 = nn.BatchNorm2d(OC)
    self.relu1 = nn.LeakyReLU()
  def forward(self, x):
    return(self.relu1(self.bn1(self.conv1(self.relu(self.bn(self.conv(x)))))))

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.conv1 = ConvBlock(3,8,3,1,1)
    self.mp1 = nn.MaxPool2d(2,2)
    self.conv2 = ConvBlock(8,16,3,1,1)
    self.mp2 = nn.MaxPool2d(2,2)
    self.conv3 = ConvBlock(16,32,3,1,1)
    self.mp3 = nn.MaxPool2d(2,2)
    self.conv4 = ConvBlock(32,64,3,1,1)
    self.mp4 = nn.MaxPool2d(2,2)
    self.conv5 = ConvBlock(64,128,3,1,1)
    self.up1 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.conv6 = ConvBlock(128,64,3,1,1)
    self.up2 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze1 = ConvBlock(128,64,3,1,1)
    self.conv7 = ConvBlock(64,32,3,1,1)
    self.up3 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze2 = ConvBlock(64,32,3,1,1)
    self.conv8 = ConvBlock(32,16,3,1,1)
    self.up4 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze3 = ConvBlock(32,16,3,1,1)
    self.conv9 = ConvBlock(16,8,3,1,1)
    self.out = nn.Conv2d(8,3,3,1,1)

  def forward(self,x):                                    # 3 * 256 * 256
    c1=self.conv1(x)                                      # 8 * 256 * 256
    cm1=self.mp1(c1)                                      # 8 * 128 * 128
    c2=self.conv2(cm1)                                    # 16 * 128 * 128
    cm2 =self.mp2(c2)                                     # 16 * 64 * 64
    c3=self.conv3(cm2)                                    # 32 * 64 * 64
    cm3 = self.mp3(c3)                                    # 32 * 32 * 32
    c4=self.conv4(cm3)                                    # 64 * 32 * 32
    cm4 = self.mp4(c4)                                    # 64 * 16 * 16
    mid = self.conv5(cm4)                                 # 128 * 16 * 16
    u1 = self.up1(mid)                                    # 128 * 32 * 32
    uc1 = torch.cat((self.conv6(u1),c4), dim=1)           # 128 * 32 * 32
    uc1s = self.squeeze1(uc1)                             # 64 * 32 * 32
    u2 = self.up2(uc1s)                                   # 64 * 64 * 64
    uc2 = torch.cat((self.conv7(u2), c3), dim=1)          # 64 * 64 * 64
    uc2s = self.squeeze2(uc2)                             # 32 * 64 * 64
    u3 = self.up3(uc2s)                                   # 32 * 128 * 128
    uc3 = torch.cat((self.conv8(u3), c2), dim=1)          # 32 * 128 * 128
    uc3s = self.squeeze3(uc3)                             # 16 * 128 * 128
    u4 = self.up4(uc3s)                                   # 16 * 256 * 256
    uc4 = self.conv9(u4)                                  # 8 * 256 * 256
    ucfinal = self.out(uc4)                               # 3 * 256 * 256
    #norm = ucfinal.pow(2).sum(dim=1, keepdim=True).sqrt()
    #out = (ucfinal/norm)
    return(torch.tanh(ucfinal))

#gen = Generator().to(device)


#inp = torch.randn(4,3,256,256)
#out = gen(inp)
#print(out.shape)
#save_image(inp, "inp.png")
#save_image(out, "out.png")

NameError: ignored

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

**Discriminator Block**

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,IC,OC,K,S,P):
    super(ConvBlock , self).__init__()
    self.conv = nn.Conv2d(IC,OC,K,S,P)
    self.bn = nn.BatchNorm2d(OC)
    self.relu = nn.ReLU()
  def forward(self, x):
    return(self.relu(self.bn(self.conv(x))))

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.conv1=ConvBlock(3,8,3,1,1)
    self.mp1 = nn.MaxPool2d(2,2)
    self.conv2=ConvBlock(8,16,3,1,1)
    self.mp2 = nn.MaxPool2d(2,2)
    self.conv3=ConvBlock(16,32,3,1,1)
    self.mp3 = nn.MaxPool2d(2,2)
    self.conv4=ConvBlock(32,64,3,1,1)
    self.mp4 = nn.MaxPool2d(2,2)
    self.conv5=ConvBlock(64,128,3,1,1)

    self.avgpl = nn.AvgPool2d(16,16)
    self.linear1 = nn.Linear(128,64)
    self.linear2 = nn.Linear(64,1)

  def forward(self,x):
    c1 = self.conv1(x)       # 3 * 256 * 256
    cm1 = self.mp1(c1)       # 3 * 128 * 128
    c2 = self.conv2(cm1)     # 8 * 128 * 128
    cm2 = self.mp2(c2)       # 8 * 64 * 64
    c3 = self.conv3(cm2)     # 16 * 64 * 64
    cm3 = self.mp3(c3)       # 16 * 32 * 32
    c4 = self.conv4(cm3)     # 32 * 32 * 32
    cm4 = self.mp4(c4)       # 32 * 16 * 16
    c5 = self.conv5(cm4)     # 64 * 16 * 16
    avg = self.avgpl(c5).squeeze() # 128 * 1 * 1
    l1 = self.linear1(avg)   # 64 * 1 * 1
    l2 = self.linear2(l1)    # 1
    return(l2)

#disc = Discriminator().to(device)
#inp = torch.randn(4,3,256,256).to(device)
#out = disc(inp)
#print(out.shape)
#save_image(inp, "inp.png")
#save_image(out, "out.png")

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
gen = Generator().to(device)
disc = Discriminator().to(device)

optGen = optim.Adam(gen.parameters())
optDisc = optim.Adam(disc.parameters())
critDisc = nn.BCEWithLogitsLoss()
critGen = nn.MSELoss()

**Training Loop**

In [ ]:
lossWeight = 0.990

tFile = open('validationLossReport(0.990).txt', 'w')

for epoch in range(NUM_EPOCHS):
  gen.train()
  disc.train()
  totalLoss = 0
  epochLoss = 0
  perEpochLoss = 0
  l = len(trainloader)
  for i, (bI, sI) in enumerate(trainloader):
    
    if torch.cuda.is_available():
      bI, sI = bI.to(device), sI.to(device)
      #print(bI.shape, sI.shape)
    
      # Training Discriminator  
      optDisc.zero_grad()  
      discOutReal = disc(sI)
      genOutD = gen(bI)  
      discOutFake = disc(genOutD)
      lossReal = critDisc(discOutReal, torch.ones_like(discOutReal).to(device))
      lossFake = critDisc(discOutFake, torch.zeros_like(discOutFake).to(device))
      lossDisc = (lossReal+lossFake)/2.0
      lossDisc.backward(retain_graph=True)
      optDisc.step()

      # Training Generator
      optGen.zero_grad()
      genOut = gen(bI)    
      discOut = disc(genOut)
      lossDiscGen = critDisc(discOut, torch.ones_like(discOut).to(device))
      lossGen = (1-lossWeight)*lossDiscGen + lossWeight*critGen(genOut, sI)
      #lossGen = lossDiscGen + critGen(genOut, sI)  
      lossGen.backward()
      optGen.step()
      #if i%25==0:
      #print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Batch {i}/{len(trainloader)} \
      #        Loss D: {lossDisc:.4f}, loss G: {lossGen:.4f}")
      totalLoss += (lossDisc.detach() + lossGen.detach())

      torch.cuda.empty_cache()
      if(i==l-2):
        
        save_image(bI, "drive/MyDrive/validationSampleImages/Images(0.990)/%03d_blurimage.png"%epoch, nrow=8)
        save_image(sI, "drive/MyDrive/validationSampleImages/Images(0.990)/%03d_sharpimage.png"%epoch, nrow=8)
        save_image(genOutD, "drive/MyDrive/validationSampleImages/Images(0.990)/%03d_genimage.png"%epoch, nrow=8)   

  sampleLoss = totalLoss/(len(data))
  print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")
  tFile.write(f"Epoch [{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")
tFile.close()
  


# 1) Save Per sample gen loss and disc loss for every epoch


Streaming output truncated to the last 5000 lines.
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(25

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    3859 MB |    8767 MB |   79849 GB |   79845 GB |\n|       from large pool |    3853 MB |    8760 MB |   79789 GB |   79785 GB |\n|       from small pool |       6 MB |      30 MB |      59 GB |      59 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |    3859 MB |    8767 MB |   79849 GB |   79845 GB |\n|       from large pool |    3853 MB |    8760 MB |

In [ ]:
'''
2) Test loader e calculate the average difference between predicted sharp image and actual sharp image
   Forward Pass for Generator and Loss calculation.
3) Save the last test batch as images after epoch
'''

for epoch in range(NUM_EPOCHS):
  for i, (bI, sI) in enumerate(testloader):
    bI, sI = bI.to(device), sI.to(device)

    print(genOut)
    #print(critGen(genOut, sI)) 

    """

    diff =   
    
    avg_diff 
    if ()
    

    # Training Generator
    genOut = gen(bI)    
    discOut = disc(genOut)
    lossDiscGen = critDisc(discOut, torch.ones_like(discOut).to(device))
    lossGen = (1-lossWeight)*lossDiscGen + lossWeight*critGen(genOut, sI)  
    """
    save_image(inp, "inp.png")

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(lossGen,label="G")
plt.plot(lossDisc,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
X = torch.randn(4,3,240,320)
norm = X.pow(2).sum(dim=1, keepdim=True).sqrt()
Xnorm = ((X/norm)+1)/2

In [ ]:
Xnorm.min(), Xnorm.max()


(tensor(4.4703e-07), tensor(1.0000))

In [ ]:
# 1.0, 0.999, 0.990, 0.950, 0.900, 0.750, 0.500, 0.250, 0.100, 0.010, 0.001, 0